In [1]:
import matplotlib.pyplot as plt

from jax import random, lax
import jax.numpy as jnp
import numpy as np

import tensorflow as tf

import math as _math
import torch
from typing import List, Optional

# IVY CONV2D (NUMPY) (FROM IVY REPO)

In [2]:
def conv2d(x, filters, strides, padding, data_format='NHWC', dilations=1):
    filter_shape = filters.shape[0:2]
    filter_shape = list(filter_shape)
    if data_format == 'NCHW':
        x = np.transpose(x, (0, 2, 3, 1))
    if padding == 'SAME':
        x = np.pad(x, [[0, 0], [filter_shape[0]//2]*2, [filter_shape[1]//2]*2, [0, 0]])
    x_shape = x.shape
    input_dim = filters.shape[-2]
    output_dim = filters.shape[-1]
    new_h = x_shape[1] - filter_shape[0] + 1
    new_w = x_shape[2] - filter_shape[1] + 1
    new_shape = [x_shape[0], new_h, new_w] + filter_shape + [x_shape[-1]]
    # ToDo: add non-unit stride support
    new_strides = x.strides[0:1] + x.strides[1:3] + x.strides[1:3] + x.strides[-1:]
    # B x OH x OW x KH x KW x I
    sub_matrices = np.lib.stride_tricks.as_strided(x, new_shape, new_strides, writeable=False)
    # B x OH x OW x KH x KW x I x O
    sub_matrices_w_output_dim = np.tile(np.expand_dims(sub_matrices, -1), [1]*6 + [output_dim])
    # B x OH x OW x KH x KW x I x O
    mult = sub_matrices_w_output_dim * filters.reshape([1]*3 + filter_shape + [input_dim, output_dim])
    # B x OH x OW x O
    res = np.sum(mult, (3, 4, 5))
    if data_format == 'NCHW':
        return np.transpose(res, (0, 3, 1, 2))
    return res

### Reshaping with shape and stride values

In [3]:
x = np.asarray([[[1, 2, 3, 4, 5], [-1, -2, -3, -4, -5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.asarray([[[1, 0, -1], [-1,  0,  1]], 
                    [[1, 1,  1], [-1, -1, -1]]], 
                    dtype=np.float32).transpose((2, 1, 0))

x = np.lib.stride_tricks.as_strided(x, shape = (1, 1, 5, 2), strides = (40, 40, 4, 20))
filters = np.lib.stride_tricks.as_strided(filters, shape = (1, 3, 2, 2), strides = (4, 4, 12, 24))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

x strides:  (40, 40, 4, 20)
filters strides: (4, 4, 12, 24)
x shape: (1, 1, 5, 2)
filters shape: (1, 3, 2, 2)
x: 
[[[[ 1. -1.]
   [ 2. -2.]
   [ 3. -3.]
   [ 4. -4.]
   [ 5. -5.]]]]
filters: 
[[[[ 1.  1.]
   [-1. -1.]]

  [[ 0.  1.]
   [ 0. -1.]]

  [[-1.  1.]
   [ 1. -1.]]]]



# NUMPY CONV1D USING IVY'S CONV2D(NUMPY) (FROM IVY REPO)

In [4]:
x = np.asarray([[[1, 2, 3, 4, -5], [1, 2, 3, 4, 5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.asarray([[[1, 0, -1], [-1,  0,  1]], 
                    [[1, 1,  1], [-1, -1, -1]]], 
                    dtype=np.float32).transpose((2, 1, 0))

x = np.lib.stride_tricks.as_strided(x, shape = (1, 1, 5, 2), strides = (40, 40, 4, 20))
filters = np.lib.stride_tricks.as_strided(filters, shape = (1, 3, 2, 2), strides = (4, 4, 12, 24))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

res = conv2d(x, filters, 1, 'SAME')
res = np.lib.stride_tricks.as_strided(res, shape=res.shape[1:], strides=res.strides[1:])
print(f"res strides: {res.strides}\nres shape: {res.shape}\nres: \n{res}\n")

x strides:  (40, 40, 4, 20)
filters strides: (4, 4, 12, 24)
x shape: (1, 1, 5, 2)
filters shape: (1, 3, 2, 2)
x: 
[[[[ 1.  1.]
   [ 2.  2.]
   [ 3.  3.]
   [ 4.  4.]
   [-5.  5.]]]]
filters: 
[[[[ 1.  1.]
   [-1. -1.]]

  [[ 0.  1.]
   [ 0. -1.]]

  [[-1.  1.]
   [ 1. -1.]]]]

res strides: (40, 8, 4)
res shape: (1, 5, 2)
res: 
[[[  0.   0.]
  [  0.   0.]
  [  0.   0.]
  [ 10. -10.]
  [  0. -10.]]]



In [5]:
x = np.asarray([[[1, 2, 3, 4, 5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.array([[[-1, 1, 1]]], dtype=np.float32).transpose((2, 1, 0))

x = np.lib.stride_tricks.as_strided(x, shape = (1, 1, 5, 1), strides = (20, 20, 4, 20))
filters = np.lib.stride_tricks.as_strided(filters, shape = (1, 3, 1, 1), strides = (4, 4, 12, 12))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

res = conv2d(x, filters, 1, 'SAME')
res = np.lib.stride_tricks.as_strided(res, shape=res.shape[1:], strides=res.strides[1:])
print(f"res strides: {res.strides}\nres shape: {res.shape}\nres: \n{res}\n")

x strides:  (20, 20, 4, 20)
filters strides: (4, 4, 12, 12)
x shape: (1, 1, 5, 1)
filters shape: (1, 3, 1, 1)
x: 
[[[[1.]
   [2.]
   [3.]
   [4.]
   [5.]]]]
filters: 
[[[[-1.]]

  [[ 1.]]

  [[ 1.]]]]

res strides: (20, 4, 4)
res shape: (1, 5, 1)
res: 
[[[3.]
  [4.]
  [5.]
  [6.]
  [1.]]]



In [6]:
def conv1d(x, filters, strides, padding, data_format='NWC', dilations=1):
    x_shape = (1,) + x.shape
    filter_shape = (1,) + filters.shape
    x_strides = (x.strides[0],) + x.strides
    filter_strides = (filters.strides[0],) + filters.strides
    x = np.lib.stride_tricks.as_strided(x, shape=x_shape, strides=x_strides)
    filters = np.lib.stride_tricks.as_strided(filters, shape=filter_shape, strides=filter_strides)
    res = conv2d(x, filters, strides, padding, data_format, dilations)
    res = np.lib.stride_tricks.as_strided(res, shape=res.shape[1:], strides=res.strides[1:])
    return res

In [7]:
x = np.asarray([[[1, 2, 3, 4, 5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.array([[[-1, 1, 1]]], dtype=np.float32).transpose((2, 1, 0))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

res = conv1d(x, filters, 1, 'SAME')
print(f"res strides: {res.strides}\nres shape: {res.shape}\nres: \n{res}\n")

x strides:  (20, 4, 20)
filters strides: (4, 12, 12)
x shape: (1, 5, 1)
filters shape: (3, 1, 1)
x: 
[[[1.]
  [2.]
  [3.]
  [4.]
  [5.]]]
filters: 
[[[-1.]]

 [[ 1.]]

 [[ 1.]]]

res strides: (20, 4, 4)
res shape: (1, 5, 1)
res: 
[[[3.]
  [4.]
  [5.]
  [6.]
  [1.]]]



In [8]:
x = np.asarray([[[1, 2, 3, 4, -5], [1, 2, 3, 4, 5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.asarray([[[1, 0, -1], [-1,  0,  1]], 
                    [[1, 1,  1], [-1, -1, -1]]], 
                    dtype=np.float32).transpose((2, 1, 0))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

res = conv1d(x, filters, 1, 'SAME')
print(f"res strides: {res.strides}\nres shape: {res.shape}\nres: \n{res}\n")

x strides:  (40, 4, 20)
filters strides: (4, 12, 24)
x shape: (1, 5, 2)
filters shape: (3, 2, 2)
x: 
[[[ 1.  1.]
  [ 2.  2.]
  [ 3.  3.]
  [ 4.  4.]
  [-5.  5.]]]
filters: 
[[[ 1.  1.]
  [-1. -1.]]

 [[ 0.  1.]
  [ 0. -1.]]

 [[-1.  1.]
  [ 1. -1.]]]

res strides: (40, 8, 4)
res shape: (1, 5, 2)
res: 
[[[  0.   0.]
  [  0.   0.]
  [  0.   0.]
  [ 10. -10.]
  [  0. -10.]]]



# JAX CONV1D USING JAX'S BUILT-IN CONV2D FUNCTION

In [9]:
x = np.asarray([[[1, 2, 3, 4, -5], [1, 2, 3, 4, 5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.asarray([[[1, 0, -1], [-1,  0,  1]], 
                    [[1, 1,  1], [-1, -1, -1]]], 
                    dtype=np.float32).transpose((2, 1, 0))

x = np.lib.stride_tricks.as_strided(x, shape = (1, 1, 5, 2), strides = (40, 40, 4, 20))
filters = np.lib.stride_tricks.as_strided(filters, shape = (1, 3, 2, 2), strides = (4, 4, 12, 24))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

dn1 = lax.conv_dimension_numbers(x.shape, filters.shape,
                                ('NHWC', 'HWIO', 'NHWC'))
print(dn1)

res = lax.conv_general_dilated(x,    # lhs = image tensor
                               filters, # rhs = conv kernel tensor
                               (1,1),  # window strides
                               'SAME', # padding mode
                               (1,1),  # lhs/image dilation
                               (1,1),  # rhs/kernel dilation
                               dn1)     # dimension_numbers = lhs, rhs, out dimension permutation

res = np.asarray(res)
res = np.lib.stride_tricks.as_strided(res, shape=res.shape[1:], strides=res.strides[1:])
print(f"res strides: {res.strides}\nres shape: {res.shape}\nres: \n{res}\n")

x strides:  (40, 40, 4, 20)
filters strides: (4, 4, 12, 24)
x shape: (1, 1, 5, 2)
filters shape: (1, 3, 2, 2)
x: 
[[[[ 1.  1.]
   [ 2.  2.]
   [ 3.  3.]
   [ 4.  4.]
   [-5.  5.]]]]
filters: 
[[[[ 1.  1.]
   [-1. -1.]]

  [[ 0.  1.]
   [ 0. -1.]]

  [[-1.  1.]
   [ 1. -1.]]]]

ConvDimensionNumbers(lhs_spec=(0, 3, 1, 2), rhs_spec=(3, 2, 0, 1), out_spec=(0, 3, 1, 2))


res strides: (40, 8, 4)
res shape: (1, 5, 2)
res: 
[[[  0.   0.]
  [  0.   0.]
  [  0.   0.]
  [ 10. -10.]
  [  0. -10.]]]



In [10]:
x = np.asarray([[[1, 2, 3, 4, 5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.array([[[-1, 1, 1]]], dtype=np.float32).transpose((2, 1, 0))

x = np.lib.stride_tricks.as_strided(x, shape = (1, 1, 5, 1), strides = (20, 20, 4, 20))
filters = np.lib.stride_tricks.as_strided(filters, shape = (1, 3, 1, 1), strides = (4, 4, 12, 12))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

dn1 = lax.conv_dimension_numbers(x.shape, filters.shape,
                                ('NHWC', 'HWIO', 'NHWC'))
print(dn1)

res = lax.conv_general_dilated(x,    # lhs = image tensor
                               filters, # rhs = conv kernel tensor
                               (1,1),  # window strides
                               'SAME', # padding mode
                               (1,1),  # lhs/image dilation
                               (1,1),  # rhs/kernel dilation
                               dn1)     # dimension_numbers = lhs, rhs, out dimension permutation

res = np.asarray(res)
res = np.lib.stride_tricks.as_strided(res, shape=res.shape[1:], strides=res.strides[1:])
print(f"res strides: {res.strides}\nres shape: {res.shape}\nres: \n{res}\n")

x strides:  (20, 20, 4, 20)
filters strides: (4, 4, 12, 12)
x shape: (1, 1, 5, 1)
filters shape: (1, 3, 1, 1)
x: 
[[[[1.]
   [2.]
   [3.]
   [4.]
   [5.]]]]
filters: 
[[[[-1.]]

  [[ 1.]]

  [[ 1.]]]]

ConvDimensionNumbers(lhs_spec=(0, 3, 1, 2), rhs_spec=(3, 2, 0, 1), out_spec=(0, 3, 1, 2))
res strides: (20, 4, 4)
res shape: (1, 5, 1)
res: 
[[[3.]
  [4.]
  [5.]
  [6.]
  [1.]]]



# JAX CONV1D USING JAX'S BUILT-IN CONV1D FUNCTION

In [11]:
x = np.asarray([[[1, 2, 3, 4, 5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.array([[[-1, 1, 1]]], dtype=np.float32).transpose((2, 1, 0))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

dn = lax.conv_dimension_numbers(x.shape, filters.shape,
                                ('NWC', 'WIO', 'NWC'))
print(dn)

data3 = jnp.asarray(x)
kernel3 = jnp.asarray(filters)

res = lax.conv_general_dilated(x,   # lhs = image tensor
                               filters, # rhs = conv kernel tensor
                               (1,),   # window strides
                               'SAME', # padding mode
                               (1,),   # lhs/image dilation
                               (1,),   # rhs/kernel dilation
                               dn)     # dimension_numbers = lhs, rhs, out dimension permutation

res = np.asarray(res)

print(f"res strides: {res.strides}\nres shape: {res.shape}\nres: \n{res}\n")

x strides:  (20, 4, 20)
filters strides: (4, 12, 12)
x shape: (1, 5, 1)
filters shape: (3, 1, 1)
x: 
[[[1.]
  [2.]
  [3.]
  [4.]
  [5.]]]
filters: 
[[[-1.]]

 [[ 1.]]

 [[ 1.]]]

ConvDimensionNumbers(lhs_spec=(0, 2, 1), rhs_spec=(2, 1, 0), out_spec=(0, 2, 1))
res strides: (20, 4, 4)
res shape: (1, 5, 1)
res: 
[[[3.]
  [4.]
  [5.]
  [6.]
  [1.]]]



# TENSORFLOW CONV1D USING TENSORFLOW'S BUILT-IN CONV1D FUNCTION

In [12]:
x = np.asarray([[[1, 2, 3, 4, -5], [1, 2, 3, 4, 5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.array([[[1, 0, -1], [-1,  0,  1]], 
                    [[1, 1,  1], [-1, -1, -1]]], 
                    dtype=np.float32).transpose([2,1,0])

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

strides = 1
padding = 'SAME'
data_format = 'NCW'
dilations = 1

if data_format == 'NCW':
    x = tf.transpose(x, (0, 1, 2))
    
res = tf.nn.conv1d(x, filters, strides, padding, 'NWC', dilations)

if data_format == 'NCW':
    res = tf.transpose(res, (0, 1, 2))

res = np.asarray(res)

print(f"result strides: {res.strides}\nresult shape: {res.shape}\nresult: \n{res}\n")

x strides:  (40, 4, 20)
filters strides: (4, 12, 24)
x shape: (1, 5, 2)
filters shape: (3, 2, 2)
x: 
[[[ 1.  1.]
  [ 2.  2.]
  [ 3.  3.]
  [ 4.  4.]
  [-5.  5.]]]
filters: 
[[[ 1.  1.]
  [-1. -1.]]

 [[ 0.  1.]
  [ 0. -1.]]

 [[-1.  1.]
  [ 1. -1.]]]

result strides: (40, 8, 4)
result shape: (1, 5, 2)
result: 
[[[  0.   0.]
  [  0.   0.]
  [  0.   0.]
  [ 10. -10.]
  [  0. -10.]]]



In [13]:
x = np.asarray([[[1, 2, 3, 4, 5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.asarray([[[-1, 1, 1]]], dtype=np.float32).transpose((2, 1, 0))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

strides = 1
padding = 'SAME'
data_format = 'NCW'
dilations = 1

if data_format == 'NCW':
    x = tf.transpose(x, (0, 1, 2))
    
res = tf.nn.conv1d(x, filters, strides, padding, 'NWC', dilations)

if data_format == 'NCW':
    res = tf.transpose(res, (0, 1, 2))

res = np.asarray(res)

print(f"result strides: {res.strides}\nresult shape: {res.shape}\nresult: \n{res}\n")

x strides:  (20, 4, 20)
filters strides: (4, 12, 12)
x shape: (1, 5, 1)
filters shape: (3, 1, 1)
x: 
[[[1.]
  [2.]
  [3.]
  [4.]
  [5.]]]
filters: 
[[[-1.]]

 [[ 1.]]

 [[ 1.]]]

result strides: (20, 4, 4)
result shape: (1, 5, 1)
result: 
[[[3.]
  [4.]
  [5.]
  [6.]
  [1.]]]



In [14]:
x = np.asarray([[[0.], [3.], [0.]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.asarray([[[0.]], [[1.]], [[0.]]], dtype=np.float32).transpose((2, 1, 0))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

strides = 1
padding = 'SAME'
data_format = 'NCW'
dilations = 1

if data_format == 'NCW':
    x = tf.transpose(x, (0, 1, 2))
    
res = tf.nn.conv1d(x, filters, strides, padding, 'NWC', dilations)

if data_format == 'NCW':
    res = tf.transpose(res, (0, 1, 2))

res = np.asarray(res)

print(f"res strides: {res.strides}\nres shape: {res.shape}\nres: \n{res}\n")

x strides:  (12, 4, 4)
filters strides: (4, 4, 4)
x shape: (1, 1, 3)
filters shape: (1, 1, 3)
x: 
[[[0. 3. 0.]]]
filters: 
[[[0. 1. 0.]]]

res strides: (12, 12, 4)
res shape: (1, 1, 3)
res: 
[[[0. 3. 0.]]]



In [15]:
x = np.asarray([[[0.], [3.], [0.]] for _ in range(5)], dtype=np.float32).transpose((0, 2, 1))
filters = np.asarray([[[0.]], [[1.]], [[0.]]], dtype=np.float32).transpose((2, 1, 0))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

strides = 1
padding = 'SAME'
data_format = 'NCW'
dilations = 1

if data_format == 'NCW':
    x = tf.transpose(x, (0, 1, 2))
    
res = tf.nn.conv1d(x, filters, strides, padding, 'NWC', dilations)

if data_format == 'NCW':
    res = tf.transpose(res, (0, 1, 2))

res = np.asarray(res)

print(f"res strides: {res.strides}\nres shape: {res.shape}\nres: \n{res}\n")

x strides:  (12, 4, 4)
filters strides: (4, 4, 4)
x shape: (5, 1, 3)
filters shape: (1, 1, 3)
x: 
[[[0. 3. 0.]]

 [[0. 3. 0.]]

 [[0. 3. 0.]]

 [[0. 3. 0.]]

 [[0. 3. 0.]]]
filters: 
[[[0. 1. 0.]]]

res strides: (12, 12, 4)
res shape: (5, 1, 3)
res: 
[[[0. 3. 0.]]

 [[0. 3. 0.]]

 [[0. 3. 0.]]

 [[0. 3. 0.]]

 [[0. 3. 0.]]]



# TORCH CONV1D USING TORCH'S BUILT-IN CONV2D FUNCTION (FROM IVY REPO)

In [16]:
x = np.asarray([[[1, 2, 3, 4, -5], [1, 2, 3, 4, 5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.asarray([[[1, 0, -1], [-1,  0,  1]], 
                    [[1, 1,  1], [-1, -1, -1]]], 
                    dtype=np.float32).transpose((2, 1, 0))

x = np.lib.stride_tricks.as_strided(x, shape = (1, 1, 5, 2), strides = (40, 40, 4, 20))
filters = np.lib.stride_tricks.as_strided(filters, shape = (1, 3, 2, 2), strides = (4, 4, 12, 24))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

x = torch.Tensor(x)
filters = torch.Tensor(filters)

strides = 1
padding = 'SAME'
data_format = 'NHWC'
dilations = 1

filter_shape = list(filters.shape[0:2])
filters = filters.permute(3, 2, 0, 1)
if data_format == 'NHWC':
    x = x.permute(0, 3, 1, 2)
if padding == 'VALID':
    padding_list: List[int] = [0, 0]
elif padding == 'SAME':
    padding_list: List[int] = [int(_math.floor(item / 2)) for item in filter_shape]
else:
    raise Exception('Invalid padding arg {}\n'
                    'Must be one of: "VALID" or "SAME"'.format(padding))
res = torch.nn.functional.conv2d(x, filters, None, strides, padding_list, dilations)
if data_format == 'NHWC':
  res = res.permute(0, 2, 3, 1)

res = np.asarray(res)
res = np.lib.stride_tricks.as_strided(res, shape=res.shape[1:], strides=res.strides[1:])
print(f"res strides: {res.strides}\nres shape: {res.shape}\nres: \n{res}\n")

x strides:  (40, 40, 4, 20)
filters strides: (4, 4, 12, 24)
x shape: (1, 1, 5, 2)
filters shape: (1, 3, 2, 2)
x: 
[[[[ 1.  1.]
   [ 2.  2.]
   [ 3.  3.]
   [ 4.  4.]
   [-5.  5.]]]]
filters: 
[[[[ 1.  1.]
   [-1. -1.]]

  [[ 0.  1.]
   [ 0. -1.]]

  [[-1.  1.]
   [ 1. -1.]]]]

res strides: (20, 4, 20)
res shape: (1, 5, 2)
res: 
[[[  0.   0.]
  [  0.   0.]
  [  0.   0.]
  [ 10. -10.]
  [  0. -10.]]]



In [17]:
x = np.asarray([[[1, 2, 3, 4, 5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.asarray([[[-1, 1, 1]]], dtype=np.float32).transpose((2, 1, 0))

x = np.lib.stride_tricks.as_strided(x, shape = (1, 1, 5, 1), strides = (20, 20, 4, 20))
filters = np.lib.stride_tricks.as_strided(filters, shape = (1, 3, 1, 1), strides = (4, 4, 12, 12))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

x = torch.from_numpy(x)
filters = torch.from_numpy(filters)

strides = 1
padding = 'SAME'
data_format = 'NHWC'
dilations = 1

filter_shape = list(filters.shape[0:2])
filters = filters.permute(3, 2, 0, 1)
if data_format == 'NHWC':
    x = x.permute(0, 3, 1, 2)
if padding == 'VALID':
    padding_list: List[int] = [0, 0]
elif padding == 'SAME':
    padding_list: List[int] = [int(_math.floor(item / 2)) for item in filter_shape]
else:
    raise Exception('Invalid padding arg {}\n'
                    'Must be one of: "VALID" or "SAME"'.format(padding))
res = torch.nn.functional.conv2d(x, filters, None, strides, padding_list, dilations)
if data_format == 'NHWC':
  res = res.permute(0, 2, 3, 1)

res = np.asarray(res)
res = np.lib.stride_tricks.as_strided(res, shape=res.shape[1:], strides=res.strides[1:])
print(f"res strides: {res.strides}\nres shape: {res.shape}\nres: \n{res}\n")

x strides:  (20, 20, 4, 20)
filters strides: (4, 4, 12, 12)
x shape: (1, 1, 5, 1)
filters shape: (1, 3, 1, 1)
x: 
[[[[1.]
   [2.]
   [3.]
   [4.]
   [5.]]]]
filters: 
[[[[-1.]]

  [[ 1.]]

  [[ 1.]]]]

res strides: (20, 4, 20)
res shape: (1, 5, 1)
res: 
[[[3.]
  [4.]
  [5.]
  [6.]
  [1.]]]



# TORCH CONV1D USING TORCH'S BUILT-IN CONV1D FUNCTION (FROM IVY REPO)

In [18]:
x = np.asarray([[[1, 2, 3, 4, -5], [1, 2, 3, 4, 5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.asarray([[[1, 0, -1], [-1,  0,  1]], 
                    [[1, 1,  1], [-1, -1, -1]]], 
                    dtype=np.float32).transpose((2, 1, 0))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

x = torch.Tensor(x)
filters = torch.Tensor(filters)

strides = 1
padding = 'SAME'
data_format = 'NWC'
dilations = 1

filter_shape = list(filters.shape[0:1])
filters = filters.permute((2, 1, 0))
if data_format == 'NWC':
    x = x.permute((0, 2, 1))
if padding == 'VALID':
    padding_list: List[int] = [0]
elif padding == 'SAME':
    padding_list: List[int] = [_math.floor(item / 2) for item in filter_shape]
else:
    raise Exception('Invalid padding arg {}\n'
                    'Must be one of: "VALID" or "SAME"'.format(padding))
res = torch.nn.functional.conv1d(x, filters, None, strides, padding_list, dilations)
res = res.permute(0, 2, 1)

res = np.asarray(res)

print(f"res strides: {res.strides}\nres shape: {res.shape}\nres: \n{res}\n")

x strides:  (40, 4, 20)
filters strides: (4, 12, 24)
x shape: (1, 5, 2)
filters shape: (3, 2, 2)
x: 
[[[ 1.  1.]
  [ 2.  2.]
  [ 3.  3.]
  [ 4.  4.]
  [-5.  5.]]]
filters: 
[[[ 1.  1.]
  [-1. -1.]]

 [[ 0.  1.]
  [ 0. -1.]]

 [[-1.  1.]
  [ 1. -1.]]]

res strides: (40, 4, 20)
res shape: (1, 5, 2)
res: 
[[[  0.   0.]
  [  0.   0.]
  [  0.   0.]
  [ 10. -10.]
  [  0. -10.]]]



In [19]:
x = np.asarray([[[1, 2, 3, 4, 5]]], dtype=np.float32).transpose((0, 2, 1))
filters = np.asarray([[[-1, 1, 1]]], dtype=np.float32).transpose((2, 1, 0))

print(f"x strides:  {x.strides}\nfilters strides: {filters.strides}\nx shape: {x.shape}\nfilters shape: {filters.shape}\nx: \n{x}\nfilters: \n{filters}\n")

x = torch.from_numpy(x)
filters = torch.from_numpy(filters)

strides = 1
padding = 'SAME'
data_format = 'NWC'
dilations = 1

filter_shape = list(filters.shape[0:1])
filters = filters.permute((2, 1, 0))
if data_format == 'NWC':
    x = x.permute((0, 2, 1))
if padding == 'VALID':
    padding_list: List[int] = [0]
elif padding == 'SAME':
    padding_list: List[int] = [_math.floor(item / 2) for item in filter_shape]
else:
    raise Exception('Invalid padding arg {}\n'
                    'Must be one of: "VALID" or "SAME"'.format(padding))
res = torch.nn.functional.conv1d(x, filters, None, strides, padding_list, dilations)
res = res.permute(0, 2, 1)

res = np.asarray(res)

print(f"res strides: {res.strides}\nres shape: {res.shape}\nres: \n{res}\n")

x strides:  (20, 4, 20)
filters strides: (4, 12, 12)
x shape: (1, 5, 1)
filters shape: (3, 1, 1)
x: 
[[[1.]
  [2.]
  [3.]
  [4.]
  [5.]]]
filters: 
[[[-1.]]

 [[ 1.]]

 [[ 1.]]]

res strides: (20, 4, 20)
res shape: (1, 5, 1)
res: 
[[[3.]
  [4.]
  [5.]
  [6.]
  [1.]]]

